In [187]:
from mido import Message, MidiFile, MidiTrack
import numpy as np
from HMM import HiddenMarkovModel
from Utility import Utility
import random
import curses
from curses.ascii import isdigit 
import nltk

#FUNCTION DEFINITIONS

def FluteMidToDat(fname):
    mid = MidiFile(fname)
    
    dat = []
    for i, track in enumerate(mid.tracks):
        if(i == 0): # taking only the first track
            for msg in track:
                #print(str(msg))
                if(msg.type == 'note_on' and msg.velocity != 0):
                    dat.append(int(msg.note));
                
    return dat


def sequenceToMidi(seq):
    for a in range(len(seq)):
        if seq[a] == 0:
            seq[a] == -1
    seq = np.add(seq, 61)
    
    ch = 0
    veloc = 64
    mid = MidiFile()
    mid.ticks_per_beat = 120
    track = MidiTrack()
    mid.tracks.append(track)
    track.append(Message('program_change',channel = ch,  program = 73, time = 0))
    
    command_time = 0
    last_command_time = 0
    
    for ele in seq:
        track.append(Message('note_on', channel = ch,  note = ele , velocity = veloc, time = 0))
        track.append(Message('note_on', channel = ch,  note = ele , velocity = 0, time = 45))

    return mid

def unsupervised_learning2(notes2, n_states, N_iters):
    '''
    Trains an HMM using supervised learning on the file 'ron.txt' and
    prints the results.

    Arguments:
        n_states:   Number of hidden states that the HMM should have.
    '''
    genres = notes2
    
    # Train the HMM.        
    HMM = unsupervised_HMM(genres, n_states, N_iters)

    # Print the transition matrix.
    print("Transition Matrix:")
    print('#' * 70)
    for i in range(len(HMM.A)):
        print(''.join("{:<12.3e}".format(HMM.A[i][j]) for j in range(len(HMM.A[i]))))
    print('')
    print('')

    # Print the observation matrix. 
    print("Observation Matrix:  ")
    print('#' * 70)
    for i in range(len(HMM.O)):
        print(''.join("{:<12.3e}".format(HMM.O[i][j]) for j in range(len(HMM.O[i]))))
    print('')
    print('')
    return HMM.A, HMM.O, HMM

def generate_emission1(self, M):
        '''
        Generates an emission of length M, assuming that the starting state
        is chosen uniformly at random. 

        Arguments:
            M:          Length of the emission to generate.

        Returns:
            emission:   The randomly generated emission as a list.

            states:     The randomly generated states as a list.
        '''

        emission = []
        states = []

        temp_lst = list(range(self.L))
        state_ind = random.choice(temp_lst)
        temp_lst = list(range(self.D))
        emission.append(np.random.choice(temp_lst, p = self.O[state_ind]))
        states.append(state_ind)
        syls = 1
        
        while True:
            temp_lst = list(range(self.L))
            temp = np.random.choice(range(self.D), p = self.O[state_ind])
            emission.append(temp)
            state_ind = np.random.choice(temp_lst, p = self.A[state_ind])
            states.append(state_ind)
            syls += 1
            if(syls == M):
                break
        return emission, states
    
def sequence_generator(A, O, k, M):
    '''
    Generates k emissions of length M using the HMM stored in the file
    'sequence_data<n>.txt' for a given n and prints the results.

    Arguments:
        N:          File index.
        K:          Number of sequences to generate.
        M:          Length of emission to generate.
    '''


    # Generate k input sequences.
    for i in range(k):
        # Initialize an HMM.
        HMM = HiddenMarkovModel(A, O)

        # Generate a single input sequence of length m.
        emission, states = generate_emission1(HMM, M)
        
        # Print the results.
        return emission


In [206]:
#DATA IMPORT

allsongs = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
major = [1,3,5,7,9,11,13,15,17]
minor = [2,4,6,8,10,12,14,16,18]
notes = [];

workinglist = major #minor doesn't work. would need to adjust a few things. allsongs and major works

for a in range(len(workinglist)):
    notes.append(FluteMidToDat("flute_midi/andersen41-" + str(workinglist[a]) +".mid"));
    temp = np.subtract(notes[a],61).tolist();
    for r in range(len(temp)):
        if temp[r] == -1:
            temp[r] = 0;
    notes[a] = temp;

In [204]:
#TRAINING

A1, O1, HMM = unsupervised_learning2(notes, 32, 100)

Iteration: 10
Iteration: 20
Iteration: 30
Iteration: 40
Iteration: 50
Iteration: 60
Iteration: 70
Iteration: 80
Iteration: 90
Iteration: 100
Transition Matrix:
######################################################################
5.289e-02   1.108e-47   7.530e-54   7.985e-309  9.586e-52   1.040e-80   1.125e-02   3.325e-37   5.274e-01   2.297e-02   1.396e-02   1.169e-113  1.608e-94   3.683e-95   1.004e-01   1.476e-28   8.636e-128  2.502e-144  1.894e-65   2.355e-112  0.000e+00   1.631e-01   1.220e-22   8.290e-72   6.871e-02   4.944e-152  7.561e-248  3.930e-02   1.876e-82   2.038e-22   1.295e-24   6.783e-55   
8.638e-16   2.173e-02   1.159e-85   6.129e-176  2.279e-122  3.560e-42   1.510e-13   1.840e-95   5.708e-28   1.409e-01   5.516e-103  5.320e-16   3.476e-160  4.254e-86   1.192e-01   4.489e-01   1.851e-123  2.296e-156  5.471e-52   1.620e-152  4.080e-223  2.019e-83   2.687e-35   6.261e-03   3.815e-50   8.696e-104  3.345e-143  2.630e-01   8.178e-65   1.587e-25   1.503e-74   3.810e-27   

In [205]:
#SEQUENCE GENERATION

seq = sequence_generator(A1, O1, 1, 100)
print(seq)
print(len(seq))
mf = sequenceToMidi(seq)

mf.save('new_song.mid')

[8, 8, 22, 8, 10, 15, 14, 15, 17, 19, 20, 18, 19, 8, 20, 18, 20, 22, 20, 8, 12, 15, 22, 19, 18, 20, 16, 19, 17, 14, 12, 15, 14, 15, 17, 15, 20, 13, 21, 19, 25, 13, 4, 13, 2, 20, 14, 10, 22, 20, 14, 15, 14, 15, 10, 11, 12, 9, 13, 8, 15, 11, 14, 3, 10, 15, 17, 16, 29, 21, 26, 21, 26, 26, 26, 26, 17, 19, 26, 19, 25, 22, 24, 21, 26, 17, 15, 20, 8, 14, 15, 19, 14, 17, 19, 12, 8, 20, 10, 17]
100
